In [1]:
import pre_processing as pp
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [103]:
files = pp.get_files("/Users/azamkhan/github/pdiot-practical/pdiot-data/2020/")
data = pp.get_data("Chest_Left", files)

In [104]:
data, tgts, students = pp.get_data_by_datapoints(data, points_wanted = 50)
tgts = pp.convert_targets(tgts)

In [105]:
cleaned_data, targets, student_data, student_targets = pp.leave_one_out(data, tgts, students)

In [14]:
clf = RandomForestClassifier()
clf.fit(cleaned_data, targets)
score = clf.score(student_data, student_targets)
print(score)

0.5879518072289157


In [106]:
timestep = 50
x_train = np.array([[c[0:timestep], c[timestep:2*timestep], c[2*timestep:3*timestep]] for c in cleaned_data])
x_test = np.array([[c[0:timestep], c[timestep:2*timestep], c[2*timestep:3*timestep]] for c in student_data])
x_train.shape

(9407, 3, 50)

In [107]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)

In [108]:
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)

In [109]:
y_train = targets
y_test = student_targets

In [110]:
x_train.shape

(9407, 3, 50, 1)

In [111]:
x_train[0].shape

(3, 50, 1)

## CNN

In [127]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, LSTM, TimeDistributed
from tensorflow.keras.layers import Conv2D, MaxPool2D, Conv1D, MaxPooling2D, MaxPooling1D, ConvLSTM2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow import keras

import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [198]:
model = Sequential()
model.add(Conv2D(16, (2,2), activation='relu', input_shape = x_train[0].shape))
model.add(Dropout(0.2))

model.add(Conv2D(32, (2,2), activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(14, activation='softmax'))

In [199]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [200]:
history = model.fit(x_train, y_train, epochs=5, batch_size = 16, validation_data = (x_test, y_test), verbose=1)

Train on 9407 samples, validate on 415 samples
Epoch 1/5
9407/9407 [==============================] - 22s 2ms/sample - loss: 1.6368 - accuracy: 0.4520 - val_loss: 0.7899 - val_accuracy: 0.7976
Epoch 2/5
9407/9407 [==============================] - 23s 2ms/sample - loss: 1.1942 - accuracy: 0.5729 - val_loss: 0.6386 - val_accuracy: 0.8627
Epoch 3/5
9407/9407 [==============================] - 22s 2ms/sample - loss: 1.0387 - accuracy: 0.6126 - val_loss: 0.5431 - val_accuracy: 0.8843
Epoch 4/5
9407/9407 [==============================] - 21s 2ms/sample - loss: 0.9809 - accuracy: 0.6367 - val_loss: 0.5954 - val_accuracy: 0.7566
Epoch 5/5
9407/9407 [==============================] - 21s 2ms/sample - loss: 0.9422 - accuracy: 0.6544 - val_loss: 0.5310 - val_accuracy: 0.7783


## CNN using L2 w/wo Dropout

In [201]:
model = Sequential()
model.add(Conv2D(16, (2,2), activation='relu', input_shape = x_train[0].shape, kernel_regularizer=l2(0.000001), bias_regularizer=l2(0.000001)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (2,2), activation='relu', kernel_regularizer=l2(0.000001), bias_regularizer=l2(0.000001)))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=1))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_regularizer=l2(0.000001), bias_regularizer=l2(0.000001)))
model.add(Dropout(0.5))

model.add(Dense(14, activation='softmax', kernel_regularizer=l2(0.000001), bias_regularizer=l2(0.000001)))

In [202]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [203]:
history = model.fit(x_train, y_train, epochs=7, batch_size = 16, validation_data = (x_test, y_test), verbose=1)

Train on 9407 samples, validate on 415 samples
Epoch 1/7
9407/9407 [==============================] - 24s 3ms/sample - loss: 1.6374 - accuracy: 0.4529 - val_loss: 0.8002 - val_accuracy: 0.7783
Epoch 2/7
9407/9407 [==============================] - 22s 2ms/sample - loss: 1.2137 - accuracy: 0.5679 - val_loss: 0.6806 - val_accuracy: 0.7904
Epoch 3/7
9407/9407 [==============================] - 22s 2ms/sample - loss: 1.0879 - accuracy: 0.6068 - val_loss: 0.6080 - val_accuracy: 0.8289
Epoch 4/7
9407/9407 [==============================] - 23s 2ms/sample - loss: 1.0133 - accuracy: 0.6269 - val_loss: 0.5761 - val_accuracy: 0.8482
Epoch 5/7
9407/9407 [==============================] - 22s 2ms/sample - loss: 0.9663 - accuracy: 0.6368 - val_loss: 0.6315 - val_accuracy: 0.7904
Epoch 6/7
9407/9407 [==============================] - 22s 2ms/sample - loss: 0.9369 - accuracy: 0.6495 - val_loss: 0.5217 - val_accuracy: 0.8771
Epoch 7/7
9407/9407 [==============================] - 22s 2ms/sample - loss:

## 1D-CNN With LSTM

In [204]:
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape = (None, 3, 50))))
#model.add(TimeDistributed(Dropout(0.1)))

model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
#model.add(TimeDistributed(Dropout(0.2)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(14, activation='softmax'))

In [205]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [206]:
history = model.fit(x_train, y_train, epochs=5, batch_size = 16, validation_data = (x_test, y_test), verbose=1)

Train on 9407 samples, validate on 415 samples
Epoch 1/5
9407/9407 [==============================] - 75s 8ms/sample - loss: 1.3563 - accuracy: 0.5326 - val_loss: 0.7101 - val_accuracy: 0.6795
Epoch 2/5
9407/9407 [==============================] - 84s 9ms/sample - loss: 0.9187 - accuracy: 0.6695 - val_loss: 0.6372 - val_accuracy: 0.7952
Epoch 3/5
9407/9407 [==============================] - 71s 8ms/sample - loss: 0.7555 - accuracy: 0.7291 - val_loss: 0.6286 - val_accuracy: 0.7687
Epoch 4/5
9407/9407 [==============================] - 79s 8ms/sample - loss: 0.6503 - accuracy: 0.7642 - val_loss: 0.7266 - val_accuracy: 0.7663
Epoch 5/5
9407/9407 [==============================] - 69s 7ms/sample - loss: 0.5804 - accuracy: 0.7929 - val_loss: 0.7207 - val_accuracy: 0.6940


## Convert Model to TF Lite

In [26]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()

float_model_size = len(tflite_float_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

Float model size = 2667KBs.


In [27]:
f = open('har.tflite', "wb")
f.write(tflite_float_model)
f.close()

print('`har.tflite` has been downloaded')

`har.tflite` has been downloaded


## LSTM

In [207]:
lx_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2])

In [208]:
lx_test = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2])

In [209]:
lx_train[0].shape

(3, 50)

In [210]:
model = Sequential()
model.add((LSTM(100, return_sequences=True, input_shape = (3,50), kernel_regularizer=l2(0.000001), bias_regularizer=l2(0.000001))))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_regularizer=l2(0.000001), bias_regularizer=l2(0.000001)))
model.add(Dropout(0.5))

model.add(Dense(14, activation='softmax', kernel_regularizer=l2(0.000001), bias_regularizer=l2(0.000001)))
model.summary()


Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_22 (LSTM)               (None, 3, 100)            60400     
_________________________________________________________________
flatten_44 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_86 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_107 (Dropout)        (None, 100)               0         
_________________________________________________________________
dense_87 (Dense)             (None, 14)                1414      
Total params: 91,914
Trainable params: 91,914
Non-trainable params: 0
_________________________________________________________________


In [211]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [212]:
history = model.fit(lx_train, y_train, epochs=10, batch_size=16, validation_data = (lx_test, y_test),verbose=1)

Train on 9407 samples, validate on 415 samples
Epoch 1/10
9407/9407 [==============================] - 26s 3ms/sample - loss: 1.6986 - accuracy: 0.4287 - val_loss: 1.2229 - val_accuracy: 0.5590
Epoch 2/10
9407/9407 [==============================] - 18s 2ms/sample - loss: 1.3686 - accuracy: 0.5167 - val_loss: 1.1568 - val_accuracy: 0.6096
Epoch 3/10
9407/9407 [==============================] - 22s 2ms/sample - loss: 1.2626 - accuracy: 0.5532 - val_loss: 1.2271 - val_accuracy: 0.6096
Epoch 4/10
9407/9407 [==============================] - 20s 2ms/sample - loss: 1.1653 - accuracy: 0.5838 - val_loss: 1.0778 - val_accuracy: 0.6482
Epoch 5/10
9407/9407 [==============================] - 29s 3ms/sample - loss: 1.0820 - accuracy: 0.6151 - val_loss: 1.1985 - val_accuracy: 0.6458
Epoch 6/10
9407/9407 [==============================] - 23s 2ms/sample - loss: 1.0186 - accuracy: 0.6320 - val_loss: 1.2185 - val_accuracy: 0.6145
Epoch 7/10
9407/9407 [==============================] - 27s 3ms/sample 